Бинарная классификация:
1. Терминал с дефектом
2. Терминал без дефекта

В качестве модели используется трансформер.

# Загрузка библиотек

In [1]:
!pip -q install vit_pytorch linformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch import nn
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
import PIL
from torch.optim import lr_scheduler
import os
import random
from sklearn.model_selection import train_test_split
import shutil
import gc
from tqdm import tqdm
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output
from vit_pytorch.efficient import ViT
from linformer import Linformer
from torch.optim.lr_scheduler import StepLR

In [3]:
RANDOM_STATE = 42

In [4]:
seed = RANDOM_STATE
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Загрузка данных

In [12]:
!sudo apt install unar
!unar 'drive/MyDrive/sorted_data_merged.rar'

Выходные данные были обрезаны до нескольких последних строк (5000).
  sorted_data_merged/receipts/ЗНО0301571566_TS903168069-05e1da94-c5f2-4898-8217-a14426b35713.jpg  (132891 B)... OK.
  sorted_data_merged/receipts/ЗНО0301571586_TS903168077-86f0f279-1da6-425e-9000-d9d0a674fdb3.jpg  (123380 B)... OK.
  sorted_data_merged/receipts/ЗНО0301571593_TS903168182-762ca820-ff6f-4389-9208-fe5da8c8e659.jpg  (191494 B)... OK.
  sorted_data_merged/receipts/ЗНО0301571737_TS903168191-aff6ca2b-c35d-42a3-babe-14e00daf4f2b.jpg  (240333 B)... OK.
  sorted_data_merged/receipts/ЗНО0301571779_TS903168194-ef758884-591e-464d-9532-2ceab6c1e7e2.jpg  (117505 B)... OK.
  sorted_data_merged/receipts/ЗНО0301571802_TS903168193-82157d01-f6e6-49f3-8e14-14e2de89bcfc.jpg  (130805 B)... OK.
  sorted_data_merged/receipts/ЗНО0301571971_TS903168176-f67d0422-8a7c-489e-89d9-9586e50e6165.jpg  (96245 B)... OK.
  sorted_data_merged/receipts/ЗНО0301571977_TS903168276-71f39c41-e268-4031-9721-2ae845054b9c.jpg  (99370 B)... OK.
  sort

In [13]:
df = pd.read_pickle('drive/MyDrive/df_markup.pkl')
df.columns

Index(['file_name', 'quality_photo', 'terminal', 'receipt', 'terminal_damaged',
       'terminal_undamaged', 'terminal_unrecognized_defect', 'other',
       'anticlass', 'hash'],
      dtype='object')

In [14]:
df = pd.read_pickle('drive/MyDrive/df_markup.pkl')
df = df.loc[
    (df['terminal']==1) & (df['quality_photo']==1) &
     (df['terminal_unrecognized_defect']==0),
    ['file_name', 'terminal', 'terminal_damaged']
]
df['file_name'] = df['file_name'].apply(lambda x: x.split('..\\data\\')[1].replace('\\', '/'))

In [15]:
df.shape

(4084, 3)

In [16]:
from PIL import UnidentifiedImageError
for file in tqdm(df['file_name']):
    try:
        img = Image.open(file)
    except UnidentifiedImageError:
        df = df[df['file_name']!=file]

100%|██████████| 4084/4084 [00:00<00:00, 5634.90it/s]


In [17]:
df.shape

(3921, 3)

In [18]:
train_data, test_data = train_test_split(
    df, stratify=df['terminal_damaged'],
    test_size=0.25
  )

In [19]:
# Augment train data
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

# Don't augment test data, only reshape
test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [20]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, target_column, transform=None):
        self.data = dataframe
        self.transform = transform
        self.target_column = target_column

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.data.iloc[idx][self.target_column]
        img = Image.open(self.data.iloc[idx]['file_name']).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

In [21]:
NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 64

train_dataset = CustomDataset(train_data, target_column='terminal_damaged', transform=train_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

test_dataset = CustomDataset(test_data, target_column='terminal_damaged', transform=test_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

# Efficient Attention

## Linformer

In [36]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=65,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

## Visual Transformer

In [37]:
model = ViT(
    dim=128,
    image_size=256,
    patch_size=32,
    num_classes=2,
    transformer=efficient_transformer,
    channels=3,
).to(device)

# Training

In [38]:
# Training settings
batch_size = 32
epochs = 20
lr = 3e-5
gamma = 0.7
seed = 42

In [39]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [41]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    epoch_f1 = 0

    for data, label in tqdm(train_dataloader, desc='Train: '):
        label = label.type(torch.LongTensor)
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        f1 = f1_score(label.cpu().numpy(), output.argmax(dim=1).cpu().numpy(), average='macro')

        epoch_f1 += f1 / len(train_dataloader)
        epoch_accuracy += acc / len(train_dataloader)
        epoch_loss += loss / len(train_dataloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        epoch_val_f1 = 0
        for data, label in tqdm(test_dataloader, desc='Test: '):
            label = label.type(torch.LongTensor)
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            f1 = f1_score(label.cpu().numpy(), val_output.argmax(dim=1).cpu().numpy(), average='macro')
            epoch_val_accuracy += acc / len(test_dataloader)
            epoch_val_f1 += f1 / len(test_dataloader)
            epoch_val_loss += val_loss / len(test_dataloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )
    print(f"f1_train:\t{epoch_f1}\tf1_val:{epoch_val_f1}\n")

Test: 100%|██████████| 16/16 [00:27<00:00,  1.73s/it]


Epoch : 1 - loss : 0.5855 - acc: 0.7231 - val_loss : 0.5867 - val_acc: 0.7184

f1_train:	0.41931967605326903	f1_val:0.4176752674552263



Test: 100%|██████████| 16/16 [00:26<00:00,  1.65s/it]


Epoch : 2 - loss : 0.5744 - acc: 0.7231 - val_loss : 0.5872 - val_acc: 0.7184

f1_train:	0.4188408986226147	f1_val:0.4171827884570492



Test: 100%|██████████| 16/16 [00:27<00:00,  1.71s/it]


...

Epoch : 18 - loss : 0.2640 - acc: 0.8933 - val_loss : 0.8056 - val_acc: 0.6997

f1_train:	0.8568057792319761	f1_val:0.6263857250408307



Test: 100%|██████████| 16/16 [00:27<00:00,  1.71s/it]


Epoch : 19 - loss : 0.2435 - acc: 0.9000 - val_loss : 0.7717 - val_acc: 0.6891

f1_train:	0.8682479941781156	f1_val:0.619214112113869



Test: 100%|██████████| 16/16 [00:28<00:00,  1.77s/it]

Epoch : 20 - loss : 0.2049 - acc: 0.9255 - val_loss : 0.9761 - val_acc: 0.6921

f1_train:	0.9020570092468378	f1_val:0.615394605084994

